In [2]:
# Define parse_cards function first
def parse_cards(card_strs):
    rank_mapping = {'2': 2, '3': 3, '4': 4, '5': 5,
                    '6': 6, '7': 7, '8': 8, '9': 9,
                    'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    suits = {'H', 'D', 'C', 'S'}
    cards = []
    for card_str in card_strs.strip().split():
        rank_char = card_str[:-1].upper()
        suit_char = card_str[-1].upper()
        if rank_char in rank_mapping and suit_char in suits:
            cards.append((rank_mapping[rank_char], suit_char))
        else:
            print(f"Invalid card: {card_str}")
    return cards

import torch
from enum import Enum

from modules import Action, MAX_FEATURE_LENGTH
# Initialize neural networks
from modules import PolicyNetwork
policy_net = PolicyNetwork(input_size=MAX_FEATURE_LENGTH, action_space=len(Action))
from modules import ValueNetwork
value_net = ValueNetwork(input_size=MAX_FEATURE_LENGTH)

# Load the saved models
policy_net.load_state_dict(torch.load('policy_net.pth'))
value_net.load_state_dict(torch.load('value_net.pth'))

# Set networks to evaluation mode
policy_net.eval()
value_net.eval()

# Prompt for user inputs with validation
def get_float_input(prompt, default_value):
    while True:
        value = input(prompt)
        if value == '':
            print(f"No input provided. Using default value: {default_value}")
            return default_value
        try:
            return float(value)
        except ValueError:
            print("Invalid input. Please enter a valid number.")

user_hand_input = input("Enter your hand (e.g., 'AH AD' for Ace of Hearts and Ace of Diamonds): ")
community_cards_input = input("Enter community cards (e.g., '10C JD QS' or leave blank if none): ")
pot_size_input = get_float_input("Enter pot size: ", default_value=0)
player_stack_input = get_float_input("Enter your stack size: ", default_value=1000)
current_bet_input = get_float_input("Enter current bet: ", default_value=0)
observed_actions_input = input("Enter observed actions (e.g., 'CALL RAISE'): ")

# Parse inputs using the parse_cards function
player_hand = parse_cards(user_hand_input)
community_cards = parse_cards(community_cards_input)
pot_size = pot_size_input
player_stack = player_stack_input
current_bet = current_bet_input
if observed_actions_input.strip() == '':
    observed_actions = []
else:
    observed_actions = [Action[action_str.upper()] for action_str in observed_actions_input.strip().split()]

# Initialize players with the input stack size
from modules import Player
ai_player = Player(id=0, stack=player_stack)  # AI player uses the same stack size
human_player = Player(id=1, stack=player_stack)
players = [ai_player, human_player]

# Assign the hand to the human player
player_hands = {human_player.id: player_hand}

# Initialize the game
from modules import TexasHoldEm
game = TexasHoldEm(players)
game.deal_hole_cards(player_hands=player_hands)
game.board = community_cards.copy()
game.pot = pot_size
game.current_bet = current_bet

In [3]:
# Parse hand
def parse_cards(card_strs):
    rank_mapping = {'2': 2, '3': 3, '4': 4, '5': 5,
                    '6': 6, '7': 7, '8': 8, '9': 9,
                    'T': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    suits = {'H', 'D', 'C', 'S'}
    cards = []
    for card_str in card_strs.strip().split():
        rank_char = card_str[:-1].upper()
        suit_char = card_str[-1].upper()
        if rank_char in rank_mapping and suit_char in suits:
            cards.append((rank_mapping[rank_char], suit_char))
        else:
            print(f"Invalid card: {card_str}")
    return cards

player_hand = parse_cards(user_hand_input)
community_cards = parse_cards(community_cards_input)
pot_size = float(pot_size_input)
player_stack = float(player_stack_input)
current_bet = float(current_bet_input)
observed_actions = [Action[action_str.upper()] for action_str in observed_actions_input.strip().split()]

In [4]:
from modules import BeliefState
ai_belief_state = BeliefState(
    observed_actions=observed_actions.copy(),   # Actions observed so far
    public_cards=game.board.copy(),
    pot_size=game.pot
)

In [5]:
from modules import extract_features
def ai_decide_action(ai_player, game, ai_belief_state):
    # Extract features
    features = extract_features(ai_belief_state)
    if features.dim() == 1:
        features = features.unsqueeze(0)
    
    # Get action probabilities
    with torch.no_grad():
        action_probs = policy_net(features)
    action_probs = action_probs.squeeze(0)
    
    # Get available actions
    valid_actions = game.get_available_actions(ai_player)
    
    # Choose an action
    from modules import sample_action
    action = sample_action(action_probs, valid_actions)
    
    return action

In [6]:
def human_decide_action(human_player, game):
    valid_actions = game.get_available_actions(human_player)
    print(f"Your hand: {human_player.hand}")
    print(f"Community cards: {game.board}")
    print(f"Pot size: {game.pot}")
    print(f"Your stack: {human_player.stack}")
    print(f"Current bet: {game.current_bet}")
    # Map valid actions to indices
    valid_actions_list = list(enumerate([action.name for action in valid_actions]))
    print("Valid actions:")
    for idx, action_name in valid_actions_list:
        print(f"{idx}: {action_name}")
   
    action = None
    while action not in valid_actions:
        action_input = input("Enter your action (number or name): ").upper()
        try:
            # Try to parse as integer index
            if action_input.isdigit():
                idx = int(action_input)
                if 0 <= idx < len(valid_actions):
                    action = valid_actions[idx]
                else:
                    print("Invalid action index. Please choose from the valid actions.")
            else:
                # Try to parse as action name
                action = Action[action_input]
                if action not in valid_actions:
                    print("Invalid action. Please choose from the valid actions.")
        except (KeyError, ValueError):
            print("Invalid action name or index. Please try again.")
    
    # Determine raise amount if necessary
    if action in [Action.BET, Action.RAISE]:
        raise_amount = float(input("Enter raise amount: "))
        game.execute_action(human_player, action, raise_amount=raise_amount)
    else:
        game.execute_action(human_player, action)
    
    return action

In [7]:
def update_belief_states(action, game, ai_belief_state, human_belief_state):
    # Update AI's belief state
    ai_belief_state.update(
        action,
        new_public_cards=game.board.copy(),
        pot_size=game.pot
    )
    
    # Update human player's belief state if needed
    # For simplicity, we can skip this if not used

In [8]:
from modules import sample_action
def ai_suggest_action(
       player_hand,
       community_cards,
       observed_actions,
       pot_size,
       player_stack,
       player_current_bet,
       current_bet,
       valid_actions
   ):
       # Create a belief state
       belief_state = BeliefState(
           observed_actions=observed_actions.copy(),
           public_cards=community_cards.copy(),
           pot_size=pot_size
       )
       belief_state.private_cards = player_hand
       
       # Create a dummy player object
       player = Player(id=0, stack=player_stack)
       player.hand = player_hand
       player.current_bet = player_current_bet
       player.folded = False
       
       # Extract features
       features = extract_features(belief_state)
       if features.dim() == 1:
           features = features.unsqueeze(0)
       
       # Get action probabilities
       with torch.no_grad():
           action_probs = policy_net(features)
       action_probs = action_probs.squeeze(0)
       
       # Choose an action
       action = sample_action(action_probs, valid_actions)
       return action

In [9]:
# Sample inputs
from modules import sample_action, extract_features, determine_raise_amount, Action

# Input your hand
player_hand = [(14, 'H'), (14, 'D')]  # For example, Ace of Hearts and Ace of Diamonds

# Input community cards
community_cards = [(10, 'C'), (11, 'D'), (12, 'S'), (13, 'H'), (9, 'C')]  # For example

# Input observed actions (opponent's actions)
observed_actions = [Action.CALL, Action.RAISE]

# Input pot size, your stack, and current bets
pot_size = 500
player_stack = 1000
player_current_bet = 0
current_bet = 100  # The amount you need to call

# Define valid actions in the current state
valid_actions = [Action.FOLD, Action.CALL, Action.RAISE]

# Get the AI's suggested action
suggested_action = ai_suggest_action(
    player_hand=player_hand,
    community_cards=community_cards,
    observed_actions=observed_actions.copy(),
    pot_size=pot_size,
    player_stack=player_stack,
    player_current_bet=player_current_bet,
    current_bet=current_bet,
    valid_actions=valid_actions
)

print(f"AI suggests action: {suggested_action.name}")

AI suggests action: FOLD


In [18]:
# Initialize belief states
ai_belief_state = BeliefState(
    observed_actions=observed_actions.copy(),
    public_cards=game.board.copy(),
    pot_size=game.pot
)

done = False

# Initialize observed_actions list to keep track of all actions
observed_actions = observed_actions.copy()

while not done:
    current_player = game.get_current_player()
    
    if current_player == ai_player:
        action = ai_decide_action(ai_player, game, ai_belief_state)
        # Determine raise amount if necessary
        if action in [Action.BET, Action.RAISE]:
            from modules import determine_raise_amount
            raise_amount = determine_raise_amount(ai_player, game)
            game.execute_action(ai_player, action, raise_amount=raise_amount)
        else:
            game.execute_action(ai_player, action)
        print(f"AI chooses to: {action.name}")
        
        # Update belief states
        observed_actions.append(action)
        update_belief_states(action, game, ai_belief_state, None)
        
    else:
        # Update variables to reflect current game state
        player_hand = current_player.hand
        community_cards = game.board.copy()
        pot_size = game.pot
        player_stack = current_player.stack
        player_current_bet = current_player.current_bet
        current_bet = game.current_bet
        valid_actions = game.get_available_actions(human_player)
        
        # AI suggests an action to the human player
        suggested_action = ai_suggest_action(
            player_hand=player_hand,
            community_cards=community_cards.copy(),
            observed_actions=observed_actions.copy(),
            pot_size=pot_size,
            player_stack=player_stack,
            player_current_bet=player_current_bet,
            current_bet=current_bet,
            valid_actions=valid_actions
        )
        print(f"AI suggests you to: {suggested_action.name}")
        
        # Proceed based on your own decision
        action = human_decide_action(human_player, game)
        
        # Update belief states
        observed_actions.append(action)
        update_belief_states(action, game, ai_belief_state, None)
    
    # Check for end of round/game
    if game.is_round_over():
        game.progress_round()
        # Prompt user to enter community cards if necessary
        if game.betting_round == 'flop':
            community_cards_input = input("Enter flop community cards (e.g., '10C JD QS'): ")
            new_community_cards = parse_cards(community_cards_input)
            game.board.extend(new_community_cards)
            ai_belief_state.public_cards = game.board.copy()
        elif game.betting_round == 'turn':
            community_cards_input = input("Enter turn community card (e.g., 'KH'): ")
            new_community_card = parse_cards(community_cards_input)
            game.board.extend(new_community_card)
            ai_belief_state.public_cards = game.board.copy()
        elif game.betting_round == 'river':
            community_cards_input = input("Enter river community card (e.g., '9D'): ")
            new_community_card = parse_cards(community_cards_input)
            game.board.extend(new_community_card)
            ai_belief_state.public_cards = game.board.copy()
    if game.is_game_over():
        game.determine_winner()
        done = True  # End the game loop


AI suggests you to: RAISE
Your hand: [(12, 'H'), (7, 'S')]
Community cards: []
Pot size: 15.0
Your stack: 500.0
Current bet: 10.0
Valid actions:
0: FOLD
1: CALL
2: RAISE
Player 1 calls 10.0.
Player 0 raises by 20 to 30.0.
AI chooses to: RAISE
AI suggests you to: CALL
Your hand: [(12, 'H'), (7, 'S')]
Community cards: []
Pot size: 55.0
Your stack: 490.0
Current bet: 30.0
Valid actions:
0: FOLD
1: CALL
2: RAISE
Player 1 calls 20.0.
Player 0 checks.
AI chooses to: CHECK
Player 0 bets 37.5.
AI chooses to: BET
AI suggests you to: CALL
Your hand: [(12, 'H'), (7, 'S')]
Community cards: [(5, 'H'), (9, 'S'), (3, 'C'), (10, 'D')]
Pot size: 112.5
Your stack: 470.0
Current bet: 37.5
Valid actions:
0: FOLD
1: CALL
2: RAISE
Player 1 raises by 130.0 to 167.5.
Player 0 folds.
AI chooses to: FOLD
